# Resolución del problema de N-Reinas con Simulated Annealing

In [1]:
import random
# Codificación de las funciones de incremento o decremento en las diagonales y movimientos horizontales y verticales 

# Función que indica que estamos en la frontera izquierda 
def frontera_izquierda(num):
    return (num -1) % 8 == 0

# Función que indica que estamos en la frontera derecha
def frontera_derecha(num):
    return num % 8 == 0

# Función para generar un estado en el tablero: genera un número uniforme discreto entre 1 y 64 
def generar_estado():
    return random.randint(1, 64)

class Tablero: 

    # Constructor de la clase 
    def __init__(self, movimientos_invalidos = {}, posiciones_reina = set()): 

        # movimientos_invalidos = { casilla_num: # apariciones }
        self.movimientos_invalidos = movimientos_invalidos
        # Conjunto que tiene las posiciones de la reina 
        self.posiciones_reina = posiciones_reina
        # Valor del tablero 
        self.valor = len(self.posiciones_reina)

    # Función para procesar un movimiento inválido
    def process_movimiento_invalido(self, num, anula):
        if(anula): 
            if(num in self.movimientos_invalidos): 
                self.movimientos_invalidos[num] += 1
            else:
                self.movimientos_invalidos[num] = 1
        else: 
            if(num in self.movimientos_invalidos): 
                self.movimientos_invalidos[num] -= 1
            if(num in self.movimientos_invalidos and self.movimientos_invalidos[num] == 0): 
                del self.movimientos_invalidos[num]

    # Función para anular los estados en la diagonal izquierda 
    def anula_diagonal_izq(self, num, anula = True ):
        if(num == 8 or num == 57):
            return 
        # Anulamos los estados en la diagonal izquierda por arriba
        i = 1
        i9 = 9*i
        if(not frontera_izquierda(num)): 
            while(num -i9 > 0):
                self.process_movimiento_invalido(num - i9, anula)
                if(frontera_izquierda(num - i9)):
                    break
                i += 1  
                i9 = 9*i

        # Anulamos los estados en la diagonal izquierda por abajo 
        i = 1
        i9 = 9*i
        if(not frontera_derecha(num)):
            while(num + i9 < 65 ):
                self.process_movimiento_invalido(num + i9, anula)
                if(frontera_derecha(num + i9)): 
                    break
                i += 1
                i9 = 9*i


    # Función para anular los estados en la diagonal derecha
    def anula_diagonal_der(self, num, anula = True ):
        if(num == 1 or num == 64):
            return
        # Anulamos los estados en la diagonal derecha por arriba
        i = 1
        i7 = 7*i
        if(not frontera_derecha(num)): 
            while(num - i7 > 0):
                self.process_movimiento_invalido(num - i7, anula)
                if(frontera_derecha(num - i7)):
                    break
                i += 1
                i7 = 7*i
            
        # Anulamos los estados en la diagonal derecha por abajo
        i = 1
        i7 = 7*i
        if(not frontera_izquierda(num)): 
            while(num +i7 < 65):
                self.process_movimiento_invalido(num + i7, anula)
                if(frontera_izquierda(num + i7)):
                    break
                i += 1
                i7 = 7*i

    # Función para anular los estados en la horizontal
    def anula_horizontal(self, num, anula = True ):
        # Anulamos la la fila horizontal por la derecha 
        i = 1
        if(not frontera_derecha(num)):
            while(not frontera_derecha(num + i)):
                self.process_movimiento_invalido(num + i, anula)
                i += 1
            if(frontera_derecha(num + i)):
                self.process_movimiento_invalido(num + i, anula)

        # Anulamos la la fila horizontal por la izquierda
        i = 1
        if(not frontera_izquierda(num)): 
            while(not frontera_izquierda(num - i) and num - i > 0):
                self.process_movimiento_invalido(num - i, anula)
                i += 1
            if(frontera_izquierda(num - i)):
                self.process_movimiento_invalido(num - i, anula)
                

    # Función para anular los estados en la vertical
    def anula_vertical(self, num, anula = True ):
        # Anulamos la fila vertical por arriba
        i = 1
        i8 = 8*i
        if(num > 8): 
            while(num - i8 > 0):
                self.process_movimiento_invalido(num - i8, anula)
                i += 1
                i8 = 8*i
        
        # Anulamos la fila vertical por abajo
        i = 1
        i8 = 8*i
        if(num < 57): 
            while(num + i8 < 65):
                self.process_movimiento_invalido(num + i8, anula)
                i += 1
                i8 = 8*i

    # Función que agrega la reina y anula de todas las direcciones 
    def agrega_reina(self, num, anula = True): 
        if(num in self.posiciones_reina or num in self.movimientos_invalidos): 
            return
        # Anulamos las diagonales
        self.anula_diagonal_izq(num, anula)
        self.anula_diagonal_der(num, anula)
        # Anulamos las filas y columnas
        self.anula_horizontal(num, anula)
        self.anula_vertical(num, anula)
        # Anulamos el estado actual
        self.process_movimiento_invalido(num, anula)
        # Agregamos la reina
        self.posiciones_reina.add(num)
        # Actualizamos el valor 
        self.valor += 1
    
    # Función que quita la reina y anula de todas las direcciones
    def quita_reina(self, num, anula = False):
        if(not num in self.posiciones_reina):
            return 
        # Agregamos el estado actual
        self.process_movimiento_invalido(num, anula)
        # Anulamos las diagonales
        self.anula_diagonal_izq(num, anula)
        self.anula_diagonal_der(num, anula)
        # Anulamos las filas y columnas
        self.anula_horizontal(num, anula)
        self.anula_vertical(num, anula)
        # Quitamos la reina
        self.posiciones_reina.remove(num)
        # Actualizamos el valor
        self.valor -= 1

    # Función para imprimir el tablero
    def print_tablero(self):
        for i in range(1, 65):
            print("|", end = " ")
            if(i in self.posiciones_reina): 
                print("Q", end = " ")
            elif(i in self.movimientos_invalidos):
                print("X", end = " ")
            else: 
                print(" ", end = " ")
            if(i % 8 == 0):
                print("|\n")
    
    # Función extra para imprimir el tablero (es lo mismo que la anterior pero en lugar de imprimir, concatena en un string)
    def toString(self): 
        string = ""
        for i in range(1, 65):
            string += "|"
            if(i in self.posiciones_reina): 
                string += "Q"
            elif(i in self.movimientos_invalidos):
                string += "X"
            else: 
                string += " "
            if(i % 8 == 0):
                string += "|\n"
        return string
    
    # Función para comparar dos objetos de la clase en función a la longitud del conjunto de posiciones de la reina
    def __lt__(self, other):
        return len(self.posiciones_reina) < len(other.posiciones_reina)

    # Función para generar una copia del tablero 
    def copia(self):
        return Tablero(self.movimientos_invalidos.copy(), self.posiciones_reina.copy())


# Simulated Annealing para resolver el problema 

Configuración inicial 

In [2]:
# Función para generar un tablero nuevo 
def FS(tablero_inicial, id = 0 ):
    tablero = tablero_inicial.copia() 
    # Vamos a intentar agregar una reina en una posición aleatoria del tablero simepre y cuando sea válido 
    nmax = 3 
    iter = 0 
    num = random.randint(1, 64)
    while(iter < nmax and num in tablero.movimientos_invalidos):
        # Generamos una posición aleatoria (que no esté en el conjunto de posiciones inválidas)
        iter += 1 
        num = random.randint(1, 64)
    if(not num in tablero.movimientos_invalidos):
        tablero.agrega_reina(num)
    else: 
        # Vamos a remover una reina aleatoria del tablero
        num = random.choice(list(tablero.posiciones_reina))
        tablero.quita_reina(num)
    return tablero

# Función objetivo 
def F(tablero): 
    return len(tablero.posiciones_reina)

# Función para construir un tablero aleatorio con k reinas 
def construye_tablero_aleatorio(k):
    tablero = Tablero({}, set())
    while(len(tablero.posiciones_reina) < k):
        num = random.randint(1, 64)
        if(not num in tablero.movimientos_invalidos):
            tablero.agrega_reina(num)
    return tablero


# Simulated Annealing

In [58]:
# Vamos a importar el algoritmo de simlated annealing 
from simulated_annealing import Simulated_Annealing

# Variables que definiremos de acuerdo al problema
T   = 350000
IT0 = 1
I   = 5000
BETTA = 1/10

# Necesito una función continua para ir decreciendo la temperatura
def decrece_temperatura(T):
    return T - BETTA * T

E0 = construye_tablero_aleatorio(3)

'''
Método de recocido simulado
@param: T (temperatura) inicial
@param: V Estructura de datos con los vecinos
@param: F Función a (minimizar o maximizar) según se indique, es la función objetivo
@param: Fs Función que selecciona un elemento vecino a una solución del conjunto de posibles soluciones
@param: I0 Estado inicial del algoritmo
@param: IT0 la cota inferior para la temperatura, en caso de que la temperatura sea menor, el algoritmo termina
@param: E Regla con la que se actualizará la temperatura del sistema
@param: I Número de iteraciones máximo
@param: subiteraciones Subiteraciones en el for
@param: tec ("minimizar") en caso que se desee minimizar la función objetivo, ("maximizar") en caso contrario

@return: Solución que minimiza o maximiza la función objetivo (según se especifique)

'''
# Fijamos una semilla 
random.seed(17)

MOptima, prom, objeto_maximo, objeto_minimo = Simulated_Annealing(T, {1, 2, 3}, F, FS, E0, IT0, decrece_temperatura, I, 1, "maximizar")

print("La mejor solución es: ")
MOptima.print_tablero()

print("El número de reinas del mejor tablero es: ", MOptima.valor)
print("El promedio de la función objetivo es: ", prom)


La mejor solución es: 
| X | X | Q | X | X | X | X | X |

| X | X | X | X | X | X | Q | X |

| X | Q | X | X | X | X | X | X |

| X | X | X | X | X | X | X | Q |

| X | X | X | X | Q | X | X | X |

| Q | X | X | X | X | X | X | X |

| X | X | X | Q | X | X | X | X |

| X | X | X | X | X | Q | X | X |

El número de reinas del mejor tablero es:  8
El promedio de la función objetivo es:  7.8304


In [62]:
# Fijamos una semilla 
random.seed(17)

MOptima, prom, objeto_maximo, objeto_minimo = Simulated_Annealing(T, {1, 2, 3}, F, FS, E0, IT0, decrece_temperatura, I, 1, "maximizar")

print("La mejor solución es: ")
MOptima.print_tablero()

print("El número de reinas del mejor tablero es: ", MOptima.valor)
print("El promedio de la función objetivo es: ", prom)

La mejor solución es: 
| X | X | Q | X | X | X | X | X |

| X | X | X | X | X | X | Q | X |

| X | Q | X | X | X | X | X | X |

| X | X | X | X | X | X | X | Q |

| X | X | X | X | Q | X | X | X |

| Q | X | X | X | X | X | X | X |

| X | X | X | Q | X | X | X | X |

| X | X | X | X | X | Q | X | X |

El número de reinas del mejor tablero es:  8
El promedio de la función objetivo es:  7.8304


# Búsqueda Tabú

In [70]:
from BusquedaTabu import Busqueda_Tabu

# Definimos los parámetros del algoritmo
MAXITER = 1000
tamaño_lista_tabu = 10
I0 = construye_tablero_aleatorio(3)

# Función objetivo (la multiplicamos por -1 para que sea una función de maximización)
def F(tablero): 
    return -len(tablero.posiciones_reina)


MOptima, _ = Busqueda_Tabu(I0, F, FS, MAXITER, tamaño_lista_tabu)

print("La mejor solución es: ")
MOptima.print_tablero()
print("El número de reinas del mejor tablero es: ", MOptima.valor)


La mejor solución es: 
| X | X | Q | X | X | X | X | X |

| X | X | X | X | X | X | X | X |

| X | X | X | X | X | X | X | Q |

| Q | X | X | X | X | X | X | X |

| X | X | X | Q | X | X | X | X |

| X | Q | X | X | X | X | X | X |

| X | X | X | X | Q | X | X | X |

| X | X | X | X | X | X | X | X |

El número de reinas del mejor tablero es:  6
